In [2]:
import lightgbm
from sklearn.datasets import load_boston
from lleaves import Model
import timeit
import treelite
import treelite_runtime

In [3]:
X, y = load_boston(return_X_y=True)
lgbm_model = lightgbm.Booster(model_file="../tests/models/boston_housing/model.txt")
llvm_model = Model(model_json="../tests/models/boston_housing/model.json")
llvm_model.compile()
treelite_model = treelite.Model.load("../tests/models/boston_housing/model.txt", model_format="lightgbm")
treelite_model.export_lib(toolchain="gcc", libpath="/tmp/boston_model.so")
treelite_model = treelite_runtime.Predictor("//tmp/boston_model.so")

[19:05:10] ../src/compiler/ast/split.cc:26: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.


In [4]:
lgbm_preds = []
llvm_preds = []
treelite_preds = []
for i in X:
    llvm_preds.append(lgbm_model.predict([i]))
    lgbm_preds.append(llvm_model.predict([i]))
    treelite_preds.append(treelite_model.predict(treelite_runtime.DMatrix([i])))

assert lgbm_preds == llvm_preds
assert lgbm_preds == treelite_preds

In [5]:
for model in [lgbm_model, llvm_model]:
    print(model)
    t = timeit.Timer("for i in X: model.predict([i])", "from __main__ import X, model")
    print(t.timeit(100))

model = treelite_model
print(model)
t = timeit.Timer("for i in X: model.predict(treelite_runtime.DMatrix([i]))", globals=globals())
print(t.timeit(100))

4.131157376003102
0.3305543780006701
11.89655774700077
